In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Ładowanie zmiennych środowiskowych
load_dotenv()

db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_database = os.getenv("DB_DATABASE")

conn = (
    f"mssql+pyodbc://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)
engine = create_engine(conn)
print("Connection successful!")

- Temperatura powietrza
- Opady (sniegu, deszczu)
- Poziomy wod (studnie, rzeki) X
- Pokrywa śnieżna X
- Mokradla X

# Srednia temperatura od 2000 do 2024 z podziałem na miesiące

In [ ]:
# Mean temperature between 2000 and 2024
meantemp_by_year_by_month = """
WITH LiczbaMiesiecyPomiarowychNaRok AS (
    SELECT
        YEAR(d.data) AS Year,
        COUNT(DISTINCT MONTH(d.data)) AS LiczbaMiesiecyPomiarowych
    FROM
        Depesze d
    GROUP BY
        YEAR(d.data)
),
LiczbaDepeszyPerStacja AS (
    SELECT
        Stacja,
        COUNT(*) AS LiczbaDepeszy
    FROM
        Depesze d
    GROUP BY
        Stacja
)
SELECT
    YEAR(d.data) AS Year,
    MONTH(d.data) AS Month,
    AVG(TemperaturaPowietrza) AS AvgTemp
FROM
    Depesze d
WHERE
    YEAR(d.data) IN (
        SELECT Year
        FROM LiczbaMiesiecyPomiarowychNaRok
        WHERE LiczbaMiesiecyPomiarowych = 12
    ) and Stacja in (
        SELECT Stacja
        FROM LiczbaDepeszyPerStacja
        WHERE LiczbaDepeszy > 30
    )
GROUP BY
    YEAR(d.data), MONTH(d.data)
ORDER BY
    Year, Month;
"""

In [ ]:
# Mean rainfall between 2000 and 2024
mean_rainfall_by_year_by_month = """WITH LiczbaMiesiecyPomiarowychNaRok AS (
    SELECT
        YEAR(d.data) AS Year,
        COUNT(DISTINCT MONTH(d.data)) AS LiczbaMiesiecyPomiarowych
    FROM
        Depesze d
    GROUP BY
        YEAR(d.data)
),
LiczbaDepeszyPerStacja AS (
    SELECT
        Stacja,
        COUNT(*) AS LiczbaDepeszy
    FROM
        Depesze d
    GROUP BY
        Stacja
)
SELECT
    YEAR(d.data) AS Year,
    MONTH(d.data) AS Month,
    AVG(WysokoscOpadu) AS AvgRainfall
FROM
    Depesze d
WHERE
    YEAR(d.data) IN (
        SELECT Year
        FROM LiczbaMiesiecyPomiarowychNaRok
        WHERE LiczbaMiesiecyPomiarowych = 12
    ) and Stacja in (
        SELECT Stacja
        FROM LiczbaDepeszyPerStacja
        WHERE LiczbaDepeszy > 30
    )
GROUP BY
    YEAR(d.data), MONTH(d.data)
ORDER BY
    Year, Month;
"""

In [ ]:
# Pobranie danych
df = pd.read_sql_query(meantemp_by_year_by_month, engine)
df

In [ ]:
# Pobranie danych
dfr = pd.read_sql_query(mean_rainfall_by_year_by_month, engine)
dfr

In [ ]:
# Sprawdzenie i raportowanie zduplikowanych oraz brakujących wartości
print("Liczba zduplikowanych wierszy:", df.duplicated().sum())
print("Liczba brakujących wartości:", df.isnull().sum().sum())

# Obsługa duplikatów
df = df.drop_duplicates()
dfr = dfr.drop_duplicates()

# Obsługa brakujących wartości, jeśli istnieją. W tym miejscu usuwamy wiersze z brakującymi wartościami, ale można je również wypełnić.
df = df.dropna()
dfr = dfr.drop_duplicates()

# Informacja po czyszczeniu danych
print("Kształt danych po czyszczeniu:", df.shape)

# Tworzenie wykresu ```
fig = px.line(df, x='Month', y='AvgTemp', color='Year',
              title='Średnia temperatura miesięczna w ciągu lat',
              labels={
                  'Month': 'Miesiąc',
                  'AvgTemp': 'Średnia temperatura (°C)',
                  'Year': 'Rok'
              })

# Poprawa czytelności wykresu i wycentrowanie tytułu
fig.update_layout(title_x=0.5,
                  xaxis=dict(tickmode='array', tickvals=list(range(1, 13)))#,
                  # xaxis_title='Miesiąc',
                  # yaxis_title='Średnia temperatura (°C)'
                  )

# Wyświetlenie wykresu
fig.show()
plt.close()

# Porównanie globalnie średniej i globalnie mediany dla każdego miesiąca ze średnią temperatury z podziałem na rok

Średnia

In [ ]:
mean_temps_by_month = df.groupby('Month')["AvgTemp"].mean().reset_index().rename(columns={"AvgTemp": "OverallAvgTempPerMonth"})
mean_temps_by_month

In [ ]:
mean_rainfall_by_month = dfr.groupby('Month')["AvgRainfall"].mean().reset_index().rename(columns={"AvgRainfall": "OverallAvgRainfallPerMonth"})
mean_rainfall_by_month

In [ ]:
merged_df_with_mean = pd.merge(df, mean_temps_by_month, on='Month')
merged_df_with_mean['OverallAvgTempAvgDiff'] = merged_df_with_mean['AvgTemp'] - merged_df_with_mean['OverallAvgTempPerMonth']
merged_df_with_mean

In [ ]:
merged_dfr_with_mean = pd.merge(dfr, mean_rainfall_by_month, on='Month')
merged_dfr_with_mean['OverallAvgRainfallAvgDiff'] = merged_dfr_with_mean['AvgRainfall'] - merged_dfr_with_mean['OverallAvgRainfallPerMonth']
merged_dfr_with_mean

Mediana

In [ ]:
median_temps_by_month = df.groupby('Month')["AvgTemp"].median().reset_index().rename(columns={"AvgTemp": "OverallMedianTempPerMonth"})
median_temps_by_month

In [ ]:
merged_df_with_median = pd.merge(df, median_temps_by_month, on='Month')
merged_df_with_median['OverallMedianTempAvgDiff'] = merged_df_with_median['AvgTemp'] - merged_df_with_median['OverallMedianTempPerMonth']
merged_df_with_median

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg,NavigationToolbar2Tk
import matplotlib.pyplot as plt
import seaborn as sns

polish_month_names = [
    "",
    "styczeń", "luty", "marzec", "kwiecień", "maj", "czerwiec",
    "lipiec", "sierpień", "wrzesień", "październik", "listopad", "grudzień"
]

merged_df_with_mean['YearMonth'] = merged_df_with_mean['Year'].map(str) + "-" + merged_df_with_mean['Month'].map(str)
merged_dfr_with_mean['YearMonth'] = merged_dfr_with_mean['Year'].map(str) + "-" + merged_dfr_with_mean['Month'].map(str)

def draw_mean_chart():
    plt.figure(figsize=(10, 4))
    plt.bar(merged_df_with_mean['YearMonth'], merged_df_with_mean['AvgTemp'], color='red')
    plt.xticks(rotation=45)
    plt.xticks(ticks=range(0, len(merged_df_with_mean),2),
               labels=merged_df_with_mean['YearMonth'][::2])
    plt.title('Wykres słupkowy średniej temperatury według roku i miesiąca')
    plt.xlabel('Rok-Miesiąc')
    plt.ylabel('Średnia temperatura')
    return plt.gcf()

def draw_mean_rainfall_chart():
    plt.figure(figsize=(10, 4))
    plt.bar(merged_dfr_with_mean['YearMonth'], merged_dfr_with_mean['AvgRainfall'], color='blue')
    plt.xticks(rotation=45)
    plt.xticks(ticks=range(0, len(merged_dfr_with_mean),2),
               labels=merged_dfr_with_mean['YearMonth'][::2])
    plt.title('Wykres słupkowy średnich opadów według roku i miesiąca')
    plt.xlabel('Rok-Miesiąc')
    plt.ylabel('Średni opad')
    return plt.gcf()

def draw_mean_heatmap():
    pivot_mean_tab = merged_df_with_mean.pivot(index="Year", columns="Month",values="OverallAvgTempAvgDiff")
    plt.figure(figsize=(10, 8))
    hm = sns.heatmap(pivot_mean_tab, annot=True, cmap="coolwarm")
    plt.title("Mapa cieplna różnic między średnią temperaturą \na średnią globalną według miesiąca i roku")
    def on_click(event):
        if event.inaxes == hm:
            x, y = int(event.xdata), int(event.ydata)
            year = pivot_mean_tab.index[y]
            month = pivot_mean_tab.columns[x]
            value = pivot_mean_tab.iloc[y, x]
            messagebox.showinfo("Dane:", f"Rok: {year}\nMiesiąc: {polish_month_names[month]}\nWartość: {value:.3f}")
    plt.gcf().canvas.mpl_connect("button_press_event", on_click)
    return plt.gcf()

def draw_mean_rainfall_heatmap():
    pivot_rain_tab = merged_dfr_with_mean.pivot(index="Year", columns="Month",values="OverallAvgRainfallAvgDiff")
    plt.figure(figsize=(10, 8))
    hm = sns.heatmap(pivot_rain_tab, annot=True, cmap="BuPu")
    plt.title("Mapa cieplna różnic między średnim opadem \na średnią globalną opadów według miesiąca i roku")
    def on_click(event):
        if event.inaxes == hm:
            x, y = int(event.xdata), int(event.ydata)
            year = pivot_rain_tab.index[y]
            month = pivot_rain_tab.columns[x]
            value = pivot_rain_tab.iloc[y, x]
            messagebox.showinfo("Dane:", f"Rok: {year}\nMiesiąc: {polish_month_names[month]}\nWartość: {value:.3f}")
    plt.gcf().canvas.mpl_connect("button_press_event", on_click)
    return plt.gcf()

def draw_median_heatmap():
    pivot_median_tab = merged_df_with_median.pivot(index="Year", columns="Month",values="OverallMedianTempAvgDiff")
    plt.figure(figsize=(10, 8))
    hm = sns.heatmap(pivot_median_tab, annot=True, cmap="coolwarm")
    plt.title("Mapa cieplna różnic między medianą \na średnią globalną według miesiąca i roku")
    def on_click(event):
        if event.inaxes == hm:
            x, y = int(event.xdata), int(event.ydata)
            year = pivot_median_tab.index[y]
            month = pivot_median_tab.columns[x]
            value = pivot_median_tab.iloc[y, x]
            messagebox.showinfo("Dane:", f"Rok: {year}\nMiesiąc: {polish_month_names[month]}\nWartość: {value:.3f}")
    plt.gcf().canvas.mpl_connect("button_press_event", on_click)
    return plt.gcf()

def display_mean_chart():
    fig = draw_mean_chart()
    for widget in graph_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    toolbar_frame = ttk.Frame(graph_frame)
    toolbar_frame.pack(fill=tk.X)
    toolbar = NavigationToolbar2Tk(canvas, toolbar_frame)
    toolbar.update()

def display_mean_rainfall_chart():
    fig = draw_mean_rainfall_chart()
    for widget in graph_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    toolbar_frame = ttk.Frame(graph_frame)
    toolbar_frame.pack(fill=tk.X)
    toolbar = NavigationToolbar2Tk(canvas, toolbar_frame)
    toolbar.update()

def display_mean_heatmap():
    fig = draw_mean_heatmap()
    for widget in graph_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def display_mean_rainfall_heatmap():
    fig = draw_mean_rainfall_heatmap()
    for widget in graph_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def display_median_heatmap():
    fig = draw_median_heatmap()
    for widget in graph_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

root = tk.Tk()
root.title("Apka")
root.geometry("1920x1080")

control_frame = ttk.Frame(root, padding=10)
control_frame.pack(side=tk.TOP, fill=tk.X)

ttk.Button(control_frame, text="Wykres średniej temp. wg rok/mies.", command=display_mean_chart).pack(side=tk.LEFT, padx=5)
ttk.Button(control_frame, text="Wykres średnich opadów  wg rok/mies.", command=display_mean_rainfall_chart).pack(side=tk.LEFT, padx=10)
ttk.Button(control_frame, text="Mapa cieplna rożnic średniej temp.", command=display_mean_heatmap).pack(side=tk.LEFT, padx=15)
ttk.Button(control_frame, text="Mapa cieplna rożnic średnich opadów", command=display_mean_rainfall_heatmap).pack(side=tk.LEFT, padx=20)
ttk.Button(control_frame, text="Mapa cieplna rożnic mediany  \n i średniej globalnej", command=display_median_heatmap).pack(side=tk.LEFT, padx=25)

graph_frame = ttk.Frame(root, padding=10)
graph_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

root.mainloop()